In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_auto_diffy")' SchwiftyNotebook_auto_diffy

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_auto_diffy")
		SchwiftyNotebook_auto_diffy
With SwiftPM flags: []
Working in: /tmp/tmp0zvz3nd0/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information availa

In [2]:
//export
import TensorFlow
import SchwiftyNotebook_auto_diffy

In [3]:
//export
import Python

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

public let plotter = Python.import("matplotlib.pyplot")

# Data Setup

In [4]:
let carDataYCSV = "/home/ubuntu/.machine-learning/data/car_stuff/craigslist_labels.csv"
let carDataXCSV = "/home/ubuntu/.machine-learning/data/car_stuff/craigslist_x_data.csv"

In [5]:
let craigslistVehiclesFull = Python.open(carDataXCSV)
print(Python.next(craigslistVehiclesFull))
print(Python.next(craigslistVehiclesFull))
print(Python.next(craigslistVehiclesFull))

,city,manufacturer,cylinders,fuel,paint_color,county_name,state_name,year,odometer,weather

0,0,0,0,0,0,0,0,2010.0,43600.0,38.0

1,0,1,1,0,1,1,0,1999.0,-1.0,38.0



In [6]:
let craigslistVehiclesLabel = Python.open(carDataYCSV)
print(Python.next(craigslistVehiclesLabel))
print(Python.next(craigslistVehiclesLabel))
print(Python.next(craigslistVehiclesLabel))

0,11900

1,1515

2,17550



In [7]:
public let numpy = Python.import("numpy")
public let pandas = Python.import("pandas")

In [9]:
let carFeatures = numpy.loadtxt(carDataXCSV, 
                                delimiter: ",", 
                                skiprows: 1, 
                                usecols: Array(1...10), 
                                dtype: Float.numpyScalarTypes.first!)

In [17]:
let featuresTensor = TensorFloat(numpy: carFeatures) 

In [18]:
featuresTensor!.shape

▿ [1723065, 10]
  ▿ dimensions : 2 elements
    - 0 : 1723065
    - 1 : 10


In [12]:
let carPrices = numpy.loadtxt(carDataYCSV, 
                                delimiter: ",", 
                                skiprows: 0, 
                                usecols: [1], 
                                dtype: Float.numpyScalarTypes.first!)

In [19]:
let labelsTensor = TensorFloat(numpy: carPrices) 

In [20]:
labelsTensor?.shape

▿ Optional<TensorShape>
  ▿ some : [1723065]
    ▿ dimensions : 1 element
      - 0 : 1723065


In [35]:
let batchSize = 1028

struct CarBatch {
    let features: TensorFloat
    let labels: TensorFloat
}

extension CarBatch: TensorGroup {
    
    public static var _typeList: [TensorDataType] = [
        Float.tensorFlowDataType,
        Float.tensorFlowDataType
    ]
    public static var _unknownShapeList: [TensorShape?] = [nil, nil]
    public var _tensorHandles: [_AnyTensorHandle] {
        fatalError("unimplemented")
    }
    public func _unpackTensorHandles(into address: UnsafeMutablePointer<CTensorHandle>?) {
        address!.advanced(by: 0).initialize(to: features.handle._cTensorHandle)
        address!.advanced(by: 1).initialize(to: labels.handle._cTensorHandle)
    }
    public init(_owning tensorHandles: UnsafePointer<CTensorHandle>?) {
        features = Tensor(handle: TensorHandle(_owning: tensorHandles!.advanced(by: 0).pointee))
        labels = Tensor(handle: TensorHandle(_owning: tensorHandles!.advanced(by: 1).pointee))
    }
    public init<C: RandomAccessCollection>(_handles: C) where C.Element: _AnyTensorHandle {
        fatalError("unimplemented")
    }
}

extension Sequence where Element == CarBatch {
    var first: CarBatch? {
        return first(where: { _ in true })
    }
}


In [36]:
extension Dataset where Element == CarBatch {
    init(featuresTensor: TensorFloat, labelsTensor: TensorFloat) {
        self.init(elements: CarBatch(features: featuresTensor, 
                                     labels: labelsTensor))
    }
}

In [44]:
let carDataPile: Dataset<CarBatch> = Dataset(
                                    featuresTensor: featuresTensor!, 
                                    labelsTensor: labelsTensor!)
                                        .batched(batchSize)
                                        .shuffled(sampleCount: 64, randomSeed: 69, reshuffleForEachIterator: true)

In [45]:
let firstBoi = carDataPile.first!

In [66]:
let testTensor = TensorFloat(randomNormal: [6, 6])

In [139]:
let item = testTensor.split(sizes: Tensor<Int32>([2, 4]), alongAxis: 1)

In [70]:
print(type(of: item), item.count)

Array<Tensor<Float>> 2


In [71]:
item[0].shape

▿ [6, 2]
  ▿ dimensions : 2 elements
    - 0 : 6
    - 1 : 2


In [72]:
item[1].shape

▿ [6, 4]
  ▿ dimensions : 2 elements
    - 0 : 6
    - 1 : 4


In [73]:
item[1]

[[-0.56196356,  -1.1872473,  -1.5214964,  0.27851576],
 [ -0.5149781,  -1.6584402,   1.7753751,  0.94688654],
 [    0.50088, -0.10709865,    1.674265,   1.3934644],
 [ -1.0517913,  -2.0560975,   1.6873406,  0.58283204],
 [ -1.4495221, -0.08175082,  -0.0504971,  0.48830736],
 [ 0.65094787,   0.6228229,   0.5293679, -0.57191646]]


In [13]:
//export
public struct DataBunch<T> where T: TensorGroup {
    public let trainingDataset: Dataset<T>
    public let validationDataset: Dataset<T>
}

In [104]:
let testFeature = item[0].concatenated(with: TensorFloat([10000, 9999,22222,44444,22222,11111]).expandingShape(at: 1), alongAxis: 1)

In [105]:
testFeature

[[  1.1075714,  -1.9598476,     10000.0],
 [-0.80613387, -0.19722898,      9999.0],
 [ -1.3030432,  -0.8992419,     22222.0],
 [0.116056904,   1.5951332,     44444.0],
 [ 0.33837235,   1.0167856,     22222.0],
 [ 0.43895292, 0.033635564,     11111.0]]


In [112]:
(testFeature.standardDeviation(), testFeature.mean())

▿ 2 elements
  - .0 : 11752.947
  - .1 : 6666.527


In [106]:
let normalizedTestFeature = normalizeTensor(tensor: testFeature)
normalizedTestFeature

[[-0.56712747,  -0.5673885,   0.2836287],
 [ -0.5672903,  -0.5672385,  0.28354362],
 [ -0.5673326, -0.56729823,   1.3235381],
 [ -0.5672118,   -0.567086,   3.2142978],
 [ -0.5671929,  -0.5671352,   1.3235381],
 [ -0.5671844, -0.56721884,  0.37815818]]


In [107]:
let normalizedControlFeature = normalizeTensor(tensor: item[0])
normalizedControlFeature

[[  1.1441343,  -1.9054635],
 [-0.75845265,  -0.1530854],
 [  -1.252475, -0.85101974],
 [ 0.15838036,   1.6288635],
 [  0.3794042,    1.053876],
 [  0.4794004,  0.07643787]]


In [108]:
let tensorsToNormalize = testFeature.unstacked(alongAxis: 1)
tensorsToNormalize

▿ 3 elements
  - 0 : [  1.1075714, -0.80613387,  -1.3030432, 0.116056904,  0.33837235,  0.43895292]
  - 1 : [ -1.9598476, -0.19722898,  -0.8992419,   1.5951332,   1.0167856, 0.033635564]
  - 2 : [10000.0,  9999.0, 22222.0, 44444.0, 22222.0, 11111.0]


In [109]:
let normalBois = tensorsToNormalize.map { normalizeTensor(tensor: $0) }
normalBois

▿ 3 elements
  - 0 : [  1.3973678, -0.97836924,   -1.595249,  0.16646893,   0.4424587,  0.56732273]
  - 1 : [ -1.6139733, -0.10988159,  -0.7089288,   1.4195912,  0.92607105, 0.087121576]
  - 2 : [ -0.8227006, -0.82278293,  0.18283767,    2.011104,  0.18283767,  -0.7312955]


In [110]:
let normalizedTensorBoi = TensorFloat(stacking: normalBois, alongAxis: 1)
normalizedTensorBoi

[[  1.3973678,  -1.6139733,  -0.8227006],
 [-0.97836924, -0.10988159, -0.82278293],
 [  -1.595249,  -0.7089288,  0.18283767],
 [ 0.16646893,   1.4195912,    2.011104],
 [  0.4424587,  0.92607105,  0.18283767],
 [ 0.56732273, 0.087121576,  -0.7312955]]


In [111]:
(normalizedTensorBoi.standardDeviation(), normalizedTensorBoi.mean())

▿ 2 elements
  - .0 : 1.0
  - .1 : 3.3113693e-08


In [4]:
//export
public func normalizeFeatureTensor(featureTensor: TensorFloat) -> TensorFloat {
    return TensorFloat(stacking: featureTensor.unstacked(alongAxis: 1)
                                               .map { normalizeTensor(tensor: $0) }, 
                       alongAxis: 1)
}

In [115]:
let shouldBeNormal = normalizeFeatureTensor(featureTensor: testFeature)
(shouldBeNormal.standardDeviation(), shouldBeNormal.mean())

▿ 2 elements
  - .0 : 1.0
  - .1 : 3.3113693e-08


In [118]:
featuresTensor?.shape

▿ Optional<TensorShape>
  ▿ some : [1723065, 10]
    ▿ dimensions : 2 elements
      - 0 : 1723065
      - 1 : 10


In [120]:
(featuresTensor?.standardDeviation(), featuresTensor?.mean())

▿ 2 elements
  ▿ .0 : Optional<Tensor<Float>>
    - some : 45456.77
  ▿ .1 : Optional<Tensor<Float>>
    - some : 7916.378


In [121]:
let normalizedFeatures = normalizeFeatureTensor(featureTensor: featuresTensor!)

In [122]:
(normalizedFeatures.standardDeviation(), normalizedControlFeature.mean())

▿ 2 elements
  - .0 : 0.99999994
  - .1 : 3.973643e-08


In [123]:
normalizedFeatures.shape

▿ [1723065, 10]
  ▿ dimensions : 2 elements
    - 0 : 1723065
    - 1 : 10


In [127]:
let validationPercent = 0.2
let numberOfInputs = normalizedFeatures.shape[0]
let numberOfFeatures = normalizedFeatures.shape[1]
(numberOfInputs, numberOfFeatures)

▿ 2 elements
  - .0 : 1723065
  - .1 : 10


In [144]:
let validationSize = Int32(floor(validationPercent * Double(numberOfInputs)))
let trainingSize = Int32(numberOfInputs) - validationSize
(validationSize, trainingSize)

▿ 2 elements
  - .0 : 344613
  - .1 : 1378452


In [147]:
(validationSize + trainingSize, numberOfInputs)

▿ 2 elements
  - .0 : 1723065
  - .1 : 1723065


In [148]:
let splitTensors = normalizedFeatures.split(sizes: Tensor<Int32>([validationSize, trainingSize]), alongAxis: 0)

In [149]:
splitTensors[0].shape

▿ [344613, 10]
  ▿ dimensions : 2 elements
    - 0 : 344613
    - 1 : 10


In [150]:
splitTensors[1].shape

▿ [1378452, 10]
  ▿ dimensions : 2 elements
    - 0 : 1378452
    - 1 : 10


In [152]:
let splitLabelTensor = labelsTensor!.split(sizes: Tensor<Int32>([validationSize, trainingSize]), alongAxis: 0)

In [154]:
(splitLabelTensor[0].shape, splitLabelTensor[1].shape)

▿ 2 elements
  ▿ .0 : [344613]
    ▿ dimensions : 1 element
      - 0 : 344613
  ▿ .1 : [1378452]
    ▿ dimensions : 1 element
      - 0 : 1378452


In [5]:
//export
public struct UsedCarBatch {
    public let features: TensorFloat
    public let labels: TensorFloat
}

extension UsedCarBatch: TensorGroup {
    
    public static var _typeList: [TensorDataType] = [
        Float.tensorFlowDataType,
        Float.tensorFlowDataType
    ]
    public static var _unknownShapeList: [TensorShape?] = [nil, nil]
    public var _tensorHandles: [_AnyTensorHandle] {
        fatalError("unimplemented")
    }
    public func _unpackTensorHandles(into address: UnsafeMutablePointer<CTensorHandle>?) {
        address!.advanced(by: 0).initialize(to: features.handle._cTensorHandle)
        address!.advanced(by: 1).initialize(to: labels.handle._cTensorHandle)
    }
    public init(_owning tensorHandles: UnsafePointer<CTensorHandle>?) {
        features = Tensor(handle: TensorHandle(_owning: tensorHandles!.advanced(by: 0).pointee))
        labels = Tensor(handle: TensorHandle(_owning: tensorHandles!.advanced(by: 1).pointee))
    }
    public init<C: RandomAccessCollection>(_handles: C) where C.Element: _AnyTensorHandle {
        fatalError("unimplemented")
    }
}

public extension Sequence where Element == UsedCarBatch {
    var first: UsedCarBatch? {
        return first(where: { _ in true })
    }
}

In [8]:
//export
public extension Dataset where Element == UsedCarBatch {
    init(featuresTensor: TensorFloat, labelsTensor: TensorFloat) {
        self.init(elements: UsedCarBatch(features: featuresTensor, 
                                     labels: labelsTensor))
    }
}

In [21]:
//export
let carDataYCSV = "/home/ubuntu/.machine-learning/data/car_stuff/craigslist_labels.csv"
let carDataXCSV = "/home/ubuntu/.machine-learning/data/car_stuff/craigslist_x_data.csv"

public let numpy = Python.import("numpy")

func createDataSet(featureTensor: TensorFloat, 
                   labelTensor: TensorFloat, 
                   batchSize: Int) -> Dataset<UsedCarBatch> {
    return Dataset(featuresTensor: featureTensor, labelsTensor: labelTensor)
                    .batched(batchSize)
                    .shuffled(sampleCount: 64, 
                              randomSeed: 69, 
                              reshuffleForEachIterator: true)
}

public func fetchUsedCarDataBunch(validationSize: Double = 0.2,
                             batchSize: Int = 1028
                            ) -> DataBunch<UsedCarBatch> {
    let usedCarFeaturesArray = numpy.loadtxt(carDataXCSV, 
                                delimiter: ",", 
                                skiprows: 1, 
                                usecols: Array(1...10), 
                                dtype: Float.numpyScalarTypes.first!)
    let usedCarFeatureTensor = normalizeFeatureTensor(featureTensor: 
                                                      TensorFloat(numpy: usedCarFeaturesArray)!)
    
    let usedCarPrices = numpy.loadtxt(carDataYCSV, 
                                delimiter: ",", 
                                skiprows: 0, 
                                usecols: [1], 
                                dtype: Float.numpyScalarTypes.first!)
    let usedCarLabelsTensor = TensorFloat(numpy: usedCarPrices)!
    
    let numberOfUsedCars = usedCarFeatureTensor.shape[0]
    let numberOfUsedCarFeatures = usedCarFeatureTensor.shape[1]
    
    let validationDatasetSize = Int32(floor(validationSize * Double(numberOfUsedCars)))
    let trainingDataSetSize = Int32(numberOfUsedCars) - validationDatasetSize
    
    
    let splitFeatures = usedCarFeatureTensor
                                .split(sizes: Tensor<Int32>([validationDatasetSize, trainingDataSetSize]), 
                                       alongAxis: 0)
    let splitLabels = usedCarLabelsTensor
                            .split(sizes: Tensor<Int32>([validationDatasetSize, trainingDataSetSize]), 
                                   alongAxis: 0)
    
    let validationDataSet = createDataSet(featureTensor: splitFeatures[0],
                                        labelTensor: splitLabels[0], batchSize: batchSize)
    let trainingDataSet = createDataSet(featureTensor: splitFeatures[1],
                                        labelTensor: splitLabels[1], batchSize: batchSize)
    return DataBunch(trainingDataset: trainingDataSet, validationDataset: validationDataSet)
}

In [22]:
let usedCarDatabunch = fetchUsedCarDataBunch()

In [27]:
exportSchwiftyNotebook(notebookName: "batch_boi")

success
